# Triplet Loss Model Capptu 

In [115]:
import numpy as np
import tensorflow as tf
import pandas as pd
import h5py
import random
import matplotlib.pyplot as plt
import cv2
import time 
import keras.backend as K
import keras 
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.utils import plot_model
from keras.optimizers import Adam, SGD
from keras.losses import mean_absolute_error, categorical_crossentropy,mean_absolute_error
from keras.layers import Flatten, Dropout, Dense, GlobalAveragePooling2D, GlobalMaxPooling2D, Lambda, concatenate, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.models import Input,Model, Sequential
from keras.callbacks import TensorBoard,ModelCheckpoint

## Global Parameters

In [116]:
TRAINING_SIZE = 100000
TESTING_SIZE = 2000
VALIDATION_SIZE = 2000

TOTAL_IMAGES = 180000
IMAGE_SIZE = 224

BATCH_SIZE = 16#8
EPOCHS = 1
STEPS_PER_EPOCH = 1000
TRIPLET_INDEX = 0
FINISH_TRIPLET_INDEX = BATCH_SIZE*STEPS_PER_EPOCH*2


ENCODINGS_DIM = 1000


## Loading Dataset

In [117]:
arod = h5py.File('./AROD_HDF/AROD.hdf','r')
triplets = pd.read_csv('./triplets.csv').get_values()[0:TRAINING_SIZE]
training_set = triplets[:,1:4]

In [118]:
def get_triplet():
    global TRIPLET_INDEX
    triplet = training_set[TRIPLET_INDEX]
    
    a = arod['IMAGES'][triplet[0]]
    p = arod['IMAGES'][triplet[1]]
    n = arod['IMAGES'][triplet[2]]
    
    sa = arod['SCORES'][triplet[0]][0]        
    sp = arod['SCORES'][triplet[1]][0]        
    sn = arod['SCORES'][triplet[2]][0]        
  
    TRIPLET_INDEX = TRIPLET_INDEX + 1
    if TRIPLET_INDEX > FINISH_TRIPLET_INDEX:
        TRIPLET_INDEX = 0 
    return a, p, n, sa, sp, sn 

In [119]:
def Generate():
    while True:
        list_a = []
        list_p = []
        list_n = []
        label = []
        la = []
        lp=[]
        ln = []
        
        
        for i in range(BATCH_SIZE):
            a, p, n, sa, sp, sn = get_triplet()
            list_a.append(a)
            list_p.append(p)
            list_n.append(n)
            la.append([sa])
            ln.append([sn])
            lp.append([sp])
            label.append([sa,sn])
        
        
        zeromat = np.zeros((BATCH_SIZE,ENCODINGS_DIM-1))
        
        

        
        
        A = preprocess_input(np.array(list_a, dtype = 'float32'))
        B = preprocess_input(np.array(list_p, dtype = 'float32'))
        C = preprocess_input(np.array(list_n, dtype = 'float32'))
        label = np.array(label,dtype = 'float32')
        photos = [A,B,C]
        labels = [np.append(np.array(la),zeromat,axis=-1),np.append(np.array(lp),zeromat,axis=-1),np.append(np.array(ln),zeromat,axis=-1)]#[np.array(la,dtype='float32'),np.array(ln,dtype='float32')]
        yield photos, labels
        
        
        
 

In [120]:
[A,P,N], labels = Generate().next()


In [121]:
labels[0][:,0]

array([0.40476879, 0.57713735, 0.17069401, 0.36863664, 0.32091293,
       0.45853236, 0.31629398, 0.55512029, 0.13033688, 0.16054408,
       0.46841747, 0.35895741, 0.34056053, 0.5638805 , 0.62254858,
       0.68500602])

In [122]:
TRIPLET_INDEX

16

## Loss Function


$L_e (a,p,n) = [m_e + ||\Phi_a - \Phi_p||^2  - ||\Phi_a - \Phi_n||^2 ]$ 

$L_d (a,p,n) = sign (s(n) - s(a) )  [m_d + ||\Phi_a|| - ||\Phi_n|| ]  $ 

$Loss = L_d + L_e$

Where :

$\Phi_i:$ Encodings of $ith$ Image

$m_d, m_e:$ Margins to avoid Trivial loss response





In [187]:
#train_generator = Generate()
#batch = next(train_generator)
############## LOSS Function ########################### 
def identity_loss(y_true, y_pred):
    r = y_true[0] - y_pred[0]
    #return K.mean(y_pred - 0 * y_true)
    return K.sum(y_pred - 0 * y_true,axis=-1)

def Le(X):
    a, p, n = X
    m = 0.3 
    loss = K.relu(m + K.sum(K.square(a-p),axis=-1,keepdims=True) - K.sum(K.square(a-n),axis=-1,keepdims=True))
    return loss

def Ld_1(X):
    a, p, n = X
    m = 0.2
    loss = K.relu(m+ K.sqrt(K.sum(K.square(a),axis=-1,keepdims=True)) - K.sqrt(K.sum(K.square(n),axis=-1,keepdims=True)))
    return loss

def triplet_loss(y_true,y_pred):
    sa = y_true[0]
    sn = y_true[1]
    #sn = y_true[2]
    
    le = y_pred[0]
    ld = y_pred[1]
    
    return (sn - sa)*ld + le

def fake_triplet_loss(y_true,y_pred):
    sa = y_true[0]
    sn = y_true[1]
    ##sn = y_true[2]
    le = y_pred[0]
    ld = y_pred[1]
#     sa,sn = y_true
#     le,ld = y_pred
    
    
    return (sn - sa) * ld + le

def newTripletLoss (y_true,y_pred):
    #sa = #K.mean(y_true[0],axis=-1)
    #sp = #K.mean(y_true[1],axis=-1)
    #sn = #K.mean(y_true[2],axis=-1)
    
    ea = y_true[0]
    ep = y_true[1]
    en = y_true[2]
    
    return ea - ep
    
    
    



In [188]:
X = [np.random.random((1000,)),np.random.random((1000,)),np.random.random((1000,))]

In [189]:
sess = tf.Session()

In [190]:
sess.run(Le(X))

array([12.90037867])

In [191]:
np.max([0,.3 + np.sum(np.square(X[0]-X[1]))-np.sum(np.square(X[0]-X[2]))])

12.90037866799014

## Neural Net model 
<img src="./Capptu model Resnet50_tripletloss.png">














In [206]:
############## Model ########################### 
def BaseModel():
    base_model = ResNet50(weights='imagenet', include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    dense_1 = Dense(ENCODINGS_DIM,activation='sigmoid')(x)
    #normalized = Lambda(lambda  x: K.l2_normalize(x,axis=1))(dense_1)
    base_model = Model(base_model.input, dense_1, name="base_model")
    return base_model

def GetModel(base_model):
    input_1 = Input((IMAGE_SIZE,IMAGE_SIZE,3))
    input_2 = Input((IMAGE_SIZE,IMAGE_SIZE,3))
    input_3 = Input((IMAGE_SIZE,IMAGE_SIZE,3))

    r1 = base_model(input_1)
    r2 = base_model(input_2)
    r3= base_model(input_3)

    #loss_le = Lambda(Le)([r1,r2,r3])
    #loss_ld1 = Lambda(Ld_1)([r1,r2,r3])
    loss = concatenate([r1,r2,r3])
    
    
    model = Model(inputs=[input_1, input_2, input_3], outputs=[loss])    
    model.compile(loss=newTripletLoss, optimizer=Adam(lr=.1))#0.000003  default
    return model

def ToyModel():

    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3) ,kernel_initializer='random_uniform') )
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu',kernel_initializer='random_uniform'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

#    model.add(Conv2D(64, (3, 3), activation='relu'))
#    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1000, activation='sigmoid'))

    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    adam = Adam()
    return model
    
#model = GetModel(GetBaseModel())
#$model.summary()

model = GetModel(ToyModel())
model.summary()
plot_model(to_file='./Capptu model Resnet50_tripletloss.png',model=model,show_layer_names=True,show_shapes=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_58 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_59 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_60 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
sequential_20 (Sequential)      (None, 1000)         84392       input_58[0][0]                   
                                                                 input_59[0][0]                   
          

In [203]:
[A,P,N], labels = Generate().next()
outs = model.predict([A,P,N])

In [204]:
sess = tf.Session()
sess.run(identity_loss(outs,outs))

array([1502.0376, 1501.5044, 1499.3223, 1501.139 , 1499.1427, 1501.2534,
       1501.7089, 1505.4724, 1501.8417, 1502.9618, 1500.8629, 1504.9036,
       1504.9573, 1502.9055, 1505.7146, 1495.3583], dtype=float32)

In [205]:
outs

array([[0.2052389 , 0.81132483, 0.5026194 , ..., 0.8212099 , 0.52416533,
        0.868542  ],
       [0.44474813, 0.59270984, 0.3090063 , ..., 0.62385046, 0.51793474,
        0.8534364 ],
       [0.21053617, 0.7905062 , 0.51991326, ..., 0.7158628 , 0.43994832,
        0.8312026 ],
       ...,
       [0.39033273, 0.57518905, 0.34032845, ..., 0.7245835 , 0.514397  ,
        0.83174086],
       [0.22263691, 0.6517118 , 0.37178037, ..., 0.82819456, 0.49352893,
        0.8978176 ],
       [0.37366557, 0.7392899 , 0.34117013, ..., 0.811876  , 0.43593508,
        0.86374336]], dtype=float32)

## Training Model

In [201]:


#filepath="weights.best.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')  #   
tb = TensorBoard(log_dir='./logs/', write_graph=True)
#tb = TensorBoard(log_dir='./log/', histogram_freq=5, write_graph=True, write_grads=True, write_images=True) 
callbacks_list =[tb]#[] [checkpoint, tb]
history = model.fit_generator(Generate(),
                    epochs=EPOCHS,#100
                    verbose=1, 
                    workers=20,
                    steps_per_epoch=STEPS_PER_EPOCH,#5000
                              callbacks = callbacks_list
        )

ValueError: An operation has `None` for gradient. Please make sure that all of your ops have a gradient defined (i.e. are differentiable). Common ops without gradient: K.argmax, K.round, K.eval.

In [195]:
TRIPLET_INDEX

624

In [141]:
751/60


12

In [ ]:
trainedmodel = model.layers[3]
trainedmodel.summary()

In [202]:
[A,P,N], labels = Generate().next()
outs = trainedmodel.predict(A)

NameError: name 'trainedmodel' is not defined

In [ ]:
outs

# Calculating Training time 
        
        for 1000 steps
        
            batch_size: 8
            epochs: 1
            triplet index : 8080
            time 355s /5.9 min
            total images processed : 24240
        
        
        
        extrapolating to 100 000 steps
        
            batch_size: 8
            epochs: 1
            triplet index : 808000
            time 35500s /9.86 min
            total images processed : 24240000
        
        
        
        
        
        
   

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")
print("Saved model to disk")
model.save(filepath='first test tripletNN.h5py')




model.save('second_save.h5py')

json_file.close()

In [ ]:
labels